##Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf

try:
  from finta import TA
except:
  !pip install finta
  from finta import TA

try:
  from backtesting import Backtest,Strategy
  from backtesting.lib import crossover
  from backtesting.test import SMA
except:
  !pip install backtesting
  from backtesting import Backtest,Strategy
  from backtesting.lib import crossover
  from backtesting.test import SMA

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 6.6 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [2]:
ohlcv=yf.download(tickers='KO',period='5y',interval='1d',auto_adjust=True,)
display(ohlcv.head())

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,KO,KO,KO,KO,KO
Date,,,,,
2020-08-05,40.575558,40.618521,40.223250,40.300586,10498700
2020-08-06,40.798969,40.841933,40.042795,40.094354,10745000
2020-08-07,41.073952,41.314555,40.566972,40.790388,11669700
2020-08-10,41.005199,41.486398,40.953641,41.297357,11888000
2020-08-11,41.185654,41.804342,40.988018,41.641075,16531600


##Cleaning

In [3]:
ohlcv.columns=ohlcv.columns.droplevel(level=1)
ohlcv.columns.name = None # Remove the name of the columns index
display(ohlcv.head())

,Close,High,Low,Open,Volume
Date,,,,,
2020-08-05,40.575558,40.618521,40.223250,40.300586,10498700
2020-08-06,40.798969,40.841933,40.042795,40.094354,10745000
2020-08-07,41.073952,41.314555,40.566972,40.790388,11669700
2020-08-10,41.005199,41.486398,40.953641,41.297357,11888000
2020-08-11,41.185654,41.804342,40.988018,41.641075,16531600


##Strategies

### Simple SMA strategy

In [4]:
class MySmaStratergy(Strategy):
  n1=12
  n2=20
  def init(self):
    self.sma1=self.I(SMA,self.data.Close,self.n1)
    self.sma2=self.I(SMA,self.data.Close,self.n2)

  def next(self):
    if crossover(self.sma1,self.sma2):
      self.buy()
    elif crossover(self.sma2,self.sma1):
      self.position.close

In [5]:
bt1=Backtest(ohlcv,MySmaStratergy,cash=10000,exclusive_orders=True,finalize_trades=True)
stats=bt1.run()

Backtest.run:   0%|          | 0/1235 [00:00<?, ?bar/s]

In [6]:
display(stats)

,0
Start,2020-08-05 00:00:00
End,2025-08-04 00:00:00
Duration,1825 days 00:00:00
Exposure Time [%],96.095618
Equity Final [$],15833.840332
Equity Peak [$],16905.244598
Return [%],58.338403
Buy & Hold Return [%],63.380564
Return (Ann.) [%],9.667081
Volatility (Ann.) [%],17.637398


In [7]:
#bt1.plot()

###RSI Strategy

In [8]:
rsi=TA.RSI(ohlcv,14)

In [9]:
class MyRSIStrategy(Strategy):
  period=20
  def init(self):
    self.rsi=self.I(TA.RSI,self.data.df,self.period)[21:]

  def next(self):
    if self.rsi[-1]<30:
      self.buy()
    elif self.rsi[-1]>70:
      self.sell()

In [10]:
bt4=Backtest(ohlcv, MyRSIStrategy, cash=10000,   exclusive_orders=True, finalize_trades=True )
stats=bt4.run()

Backtest.run:   0%|          | 0/1254 [00:00<?, ?bar/s]

In [11]:
display(stats)

,0
Start,2020-08-05 00:00:00
End,2025-08-04 00:00:00
Duration,1825 days 00:00:00
Exposure Time [%],99.840637
Equity Final [$],3188.688188
Equity Peak [$],10040.005213
Return [%],-68.113118
Buy & Hold Return [%],69.954532
Return (Ann.) [%],-20.507367
Volatility (Ann.) [%],15.324797


In [12]:
#bt4.plot()

###Combination

In [13]:
display(TA.RSI(ohlcv,15))
display(TA.SMA(ohlcv,20))

,15 period RSI
Date,
2020-08-05,NaN
2020-08-06,100.000000
2020-08-07,100.000000
2020-08-10,86.778886
2020-08-11,90.362226
...,...
2025-07-29,47.021280
2025-07-30,43.428392
2025-07-31,39.062865


,20 period SMA
Date,
2020-08-05,NaN
2020-08-06,NaN
2020-08-07,NaN
2020-08-10,NaN
2020-08-11,NaN
...,...
2025-07-29,69.8735
2025-07-30,69.7275
2025-07-31,69.5765


In [14]:
class MyCombinationStrategy1(Strategy):
  n1=30
  n2=70
  period=14
  sl=0.93
  tgt=1.2

  def init(self):
    self.rsi=self.I(TA.RSI,self.data.df,self.period)
    self.sma1=self.I(TA.SMA,self.data.df,self.n1)
    self.sma2=self.I(TA.SMA,self.data.df,self.n2)
    self.crossover_series=[]
  def next(self):
    price = self.data.Close[-1]

    # Ensure enough history to look back 3 candles
    if len(self.sma1) < 3 or len(self.sma2) < 3:
        return

    # Check if crossover happened 2 candles ago
    if (
        self.sma1[-2] < self.sma2[-2] and  # 1 candles ago: before crossover
        self.sma1[-1] > self.sma2[-2] and  # current candles ago: crossover
        self.sma1[-1] > self.sma2[-1] and  # trend still up
        30 < self.rsi[-1] < 80             # RSI filter
        ):
      self.buy(
      sl=price * self.sl,
      tp=price * self.tgt
        )
    elif(
        (self.sma1[-2] > self.sma2[-2] and  # 3 candles ago: before crossover
        self.sma1[-1] < self.sma2[-1]) or
        (self.rsi[-1]<30 or self.rsi[-1]>80)
        ):
      self.position.close()

In [15]:
bt5=Backtest(ohlcv, MyCombinationStrategy1, cash=10000,   exclusive_orders=True, finalize_trades=True )
stats=bt5.run()

Backtest.run:   0%|          | 0/1185 [00:00<?, ?bar/s]

In [16]:
display(stats)

,0
Start,2020-08-05 00:00:00
End,2025-08-04 00:00:00
Duration,1825 days 00:00:00
Exposure Time [%],49.083665
Equity Final [$],13293.425448
Equity Peak [$],13730.683757
Return [%],32.934254
Buy & Hold Return [%],48.578064
Return (Ann.) [%],5.882917
Volatility (Ann.) [%],11.072706


In [17]:
#bt5.plot()

In [18]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,Commission,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_RSI(df,14)","Exit_RSI(df,14)","Entry_SMA(df,30)","Exit_SMA(df,30)","Entry_SMA(df,70)","Exit_SMA(df,70)"
0,215,162,290,46.418325,47.040634,43.422838,56.029468,133.796447,0.0,0.013407,2021-03-29,2021-09-29,184 days,None,72.919533,30.327540,44.555195,49.303872,44.391159,49.409902
1,202,325,431,50.078960,60.287778,46.723028,60.287778,2062.181111,0.0,0.203854,2021-11-17,2022-04-21,155 days,None,52.437012,75.292612,49.494530,56.363697,49.346831,55.580743
2,209,504,538,58.158380,54.282855,54.121293,69.833926,-809.984838,0.0,-0.066637,2022-08-05,2022-09-23,49 days,None,53.131458,30.465843,57.523447,56.966954,57.386695,57.094793
3,197,584,632,57.560452,55.557958,53.685341,69.271408,-394.491337,0.0,-0.034789,2022-11-29,2023-02-08,71 days,None,66.743431,38.107271,55.077705,56.987702,54.699172,57.139800
4,188,675,706,58.375954,56.497989,54.376534,70.163269,-353.057358,0.0,-0.032170,2023-04-12,2023-05-25,43 days,None,67.922621,24.278335,56.718982,59.159536,56.489072,57.630921
5,190,837,1004,55.732622,67.074257,51.982549,67.074257,2154.910546,0.0,0.203501,2023-12-01,2024-08-02,245 days,None,70.521843,75.813896,54.097122,62.879750,53.893275,61.516037
6,187,1138,1233,68.365825,71.040001,63.708555,82.204587,500.070878,0.0,0.039116,2025-02-14,2025-07-03,139 days,None,72.418319,55.103088,62.368044,70.722530,62.182567,70.767855


###MACD Crossover

In [19]:
class MyMACDStratergy(Strategy):
  n1=12
  n2=26
  n3=9

  def init(self):
    self.macd=self.I(TA.MACD,self.data.df,self.n1,self.n2,self.n3)[0][25:]          #TA.MACD(df,period_fast,period_slow,signal)=>[[MACD],[SIGNAL]]
    self.signal=self.I(TA.MACD,self.data.df,self.n1,self.n2,self.n3)[1][25:]

  def next(self):
    if crossover(self.macd, self.signal):
      self.buy()
    elif crossover(self.signal, self.macd):
      self.sell()

In [20]:
bt2=Backtest(ohlcv,MyMACDStratergy, cash=10000, exclusive_orders=True, finalize_trades=True )
stats=bt2.run()

Backtest.run:   0%|          | 0/1254 [00:00<?, ?bar/s]

In [21]:
display(stats)

,0
Start,2020-08-05 00:00:00
End,2025-08-04 00:00:00
Duration,1825 days 00:00:00
Exposure Time [%],99.36255
Equity Final [$],2859.779917
Equity Peak [$],10214.479135
Return [%],-71.402201
Buy & Hold Return [%],69.954532
Return (Ann.) [%],-22.2262
Volatility (Ann.) [%],13.171786


In [22]:
#bt2.plot()

###VWAP Strategy

In [23]:
display(TA.VWAP(ohlcv))

,VWAP.
Date,
2020-08-05,40.472443
2020-08-06,40.517353
2020-08-07,40.683217
2020-08-10,40.806656
2020-08-11,40.946641
...,...
2025-07-29,56.053961
2025-07-30,56.061029
2025-07-31,56.071937


In [24]:
class MyVWAPStrategy(Strategy):
  def init(self):
    self.vwap=self.I(TA.VWAP,self.data.df)

  def next(self):
    if crossover(self.data.Close,self.vwap):
      self.buy()
    elif crossover(self.vwap,self.data.Open):
      self.sell()

In [25]:
bt3=Backtest(ohlcv,MyVWAPStrategy, cash=10000,   exclusive_orders=True, finalize_trades=True )
stats=bt3.run()

Backtest.run:   0%|          | 0/1254 [00:00<?, ?bar/s]

In [26]:
display(stats)

,0
Start,2020-08-05 00:00:00
End,2025-08-04 00:00:00
Duration,1825 days 00:00:00
Exposure Time [%],99.043825
Equity Final [$],10943.714868
Equity Peak [$],11684.381295
Return [%],9.437149
Buy & Hold Return [%],69.954532
Return (Ann.) [%],1.827284
Volatility (Ann.) [%],16.902567


In [27]:
#bt3.plot()

###Combination2

In [28]:
display(TA.MACD(ohlcv,12,26,9))
display(TA.WMA(ohlcv,20))

,MACD,SIGNAL
Date,,
2020-08-05,0.000000,0.000000
2020-08-06,0.005012,0.002785
2020-08-07,0.014914,0.007756
2020-08-10,0.016570,0.010741
2020-08-11,0.024301,0.014775
...,...,...
2025-07-29,-0.385890,-0.299796
2025-07-30,-0.408504,-0.321537
2025-07-31,-0.490169,-0.355264


,20 period WMA.
Date,
2020-08-05,NaN
2020-08-06,NaN
2020-08-07,NaN
2020-08-10,NaN
2020-08-11,NaN
...,...
2025-07-29,69.506904
2025-07-30,69.399904
2025-07-31,69.224904


In [29]:
class MyCombinationStrategy2(Strategy):
  n1=12
  n2=26
  n3=9
  sl=0.96

  def init(self):
    self.macd=self.I(TA.MACD,self.data.df,self.n1,self.n2,self.n3)[0]         #TA.MACD(df,period_fast,period_slow,signal)=>[[MACD],[SIGNAL]]
    self.signal=self.I(TA.MACD,self.data.df,self.n1,self.n2,self.n3)[1]
    self.wma=self.I(TA.WMA,self.data.df)[25:]

  def next(self):
    self.price=self.data.Close[-1]
    if (self.macd <0.3)and (self.signal<0.3) and crossover(self.macd,self.signal):
      self.buy(
          sl=self.price*self.sl
      )
    elif (self.macd >0)and (self.signal>0) and crossover(self.signal,self.macd):
      self.position.close()

In [30]:
bt6=Backtest(ohlcv, MyCombinationStrategy2, cash=10000, exclusive_orders=True, finalize_trades=True )
stats=bt6.run()

Backtest.run:   0%|          | 0/1254 [00:00<?, ?bar/s]

In [31]:
display(stats)

,0
Start,2020-08-05 00:00:00
End,2025-08-04 00:00:00
Duration,1825 days 00:00:00
Exposure Time [%],45.737052
Equity Final [$],18363.73896
Equity Peak [$],20121.098778
Return [%],83.63739
Buy & Hold Return [%],69.954532
Return (Ann.) [%],12.980255
Volatility (Ann.) [%],11.473441


In [32]:
#bt6.plot()

In [33]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,Commission,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_MACD(df,12,26,9)_0","Exit_MACD(df,12,26,9)_0","Entry_MACD(df,12,26,9)_1","Exit_MACD(df,12,26,9)_1",Entry_WMA(df),Exit_WMA(df)
0,238,17,33,41.890266,42.541214,39.785698,None,154.925465,0.0,0.015539,2020-08-28,2020-09-22,25 days,None,0.118217,0.352246,0.028438,0.440302,41.495376,43.452469
1,231,46,53,43.892540,43.216873,41.962207,None,-156.079132,0.0,-0.015394,2020-10-09,2020-10-20,11 days,None,0.169462,0.181348,0.084337,0.182152,43.140259,43.431316
2,226,56,58,44.178404,43.147582,42.145155,None,-232.965767,0.0,-0.023333,2020-10-23,2020-10-27,4 days,None,0.217490,0.149880,0.190843,0.180415,43.508508,43.409950
3,214,68,81,45.538401,45.476411,43.716863,None,-13.265898,0.0,-0.001361,2020-11-10,2020-11-30,20 days,None,0.383859,0.604351,0.022913,0.706267,43.936240,45.657590
4,228,125,173,42.735085,46.638397,41.034053,None,889.955148,0.0,0.091337,2021-02-03,2021-04-14,70 days,None,-0.765586,0.608986,-0.819145,0.636724,42.539327,46.781008
5,218,235,252,48.737082,50.058611,46.387412,None,288.093122,0.0,0.027115,2021-07-13,2021-08-05,23 days,None,-0.019294,0.433517,-0.065830,0.492990,48.263260,50.302423
6,226,297,325,48.255959,50.078960,46.222775,None,411.998300,0.0,0.037778,2021-10-08,2021-11-17,40 days,None,-0.460217,0.421119,-0.553311,0.472725,47.770820,50.410598
7,230,340,368,49.305601,54.934097,47.549509,None,1294.554163,0.0,0.114155,2021-12-09,2022-01-20,42 days,None,-0.132356,0.975850,-0.198217,1.032570,48.942572,54.805812
8,228,410,438,55.291077,59.238495,52.774530,None,900.011427,0.0,0.071393,2022-03-22,2022-05-02,41 days,None,-0.108942,0.753574,-0.199904,0.965611,54.384236,59.038253
9,232,458,465,58.149557,56.346270,56.346270,None,-418.362665,0.0,-0.031011,2022-05-31,2022-06-09,9 days,None,-0.076054,-0.296002,-0.070007,-0.170908,57.634527,57.062831
